# Carga/Consumo

Primeiro modelo (não utilizado)

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

start_date = '2019-01-01'
end_date = datetime.now()

df_carga = pd.DataFrame()

def obter_carga_verificada(api_url, data_inicio, data_fim, cod_areacarga):
    url = f'{api_url}?dat_inicio={data_inicio}&dat_fim={data_fim}&cod_areacarga={cod_areacarga}'
    
    try:
        resposta = requests.get(url)
        resposta.raise_for_status()  

        dados = resposta.json()
        
        return dados
    except requests.exceptions.HTTPError as errh:
        print(f"Erro HTTP: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Erro de Conexão: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Tempo de Conexão Esgotado: {errt}")
    except requests.exceptions.RequestException as err:
        print(f"Erro ao fazer a requisição: {err}")
    
    return None  

api_url = 'https://apicarga.ons.org.br/prd/cargaverificada'

def generate_date_ranges(start_date_str, finish_date):
    start_date = datetime.strptime(start_date_str, '%Y-%m-%d')

    current_start_date = start_date

    while True:
        current_finish_date = current_start_date + timedelta(days=90) 

        end = current_finish_date.strftime('%Y-%m-%d')

        if current_finish_date > finish_date:
            break

        dados_carga = obter_carga_verificada(api_url, current_start_date.strftime('%Y-%m-%d'), end, cod_areacarga)
        
        if dados_carga is not None:
            df_temp = pd.DataFrame(dados_carga) 
            global df_carga  
            df_carga = pd.concat([df_carga, df_temp], ignore_index=True)

        current_start_date = current_finish_date



def values_treatment (df_carga):
    df_carga = df_carga[df_carga['val_cargaglobal'] != 0]
    df_carga = df_carga.drop_duplicates()
    df_carga['din_referenciautc'] = pd.to_datetime(df_carga['din_referenciautc'])

    df_carga['year'] = df_carga['din_referenciautc'].dt.year
    df_carga['month'] = df_carga['din_referenciautc'].dt.month
    df_carga['day'] = df_carga['din_referenciautc'].dt.day
    df_carga['hour'] = df_carga['din_referenciautc'].dt.hour

    result = df_carga.groupby(['year', 'month', 'day', 'hour'], as_index=False).agg(
        val_cargaglobal=('val_cargaglobal', 'mean'),
        val_cargaglobalcons=('val_cargaglobalcons', 'mean'),
        val_cargaglobalsmmgd=('val_cargaglobalsmmgd', 'mean'),
        val_cargasupervisionada=('val_cargasupervisionada', 'mean'),
        val_carganaosupervisionada=('val_carganaosupervisionada', 'mean'),
        val_cargammgd=('val_cargammgd', 'mean'),
        val_consistencia=('val_consistencia', 'mean'),
        cod_areacarga=('cod_areacarga', 'first'),  
        din_atualizacao=('din_atualizacao', 'first'),    
        dat_referencia=('dat_referencia', 'first')  
    )

    result['din_referenciautc'] = pd.to_datetime(result[['year', 'month', 'day', 'hour']].assign(minute=0))

    result['din_referenciautc'] = result['din_referenciautc'].dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')

    result = result.drop(columns=['year', 'month', 'day', 'hour'])

    return result
cod_areacarga = 'SECO'
generate_date_ranges(start_date, end_date)
resultSECO = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'NE'
generate_date_ranges(start_date, end_date)
resultNE = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'N'
generate_date_ranges(start_date, end_date)
resultN = values_treatment(df_carga)

df_carga = pd.DataFrame()
cod_areacarga = 'S'
generate_date_ranges(start_date, end_date)
resultS = values_treatment(df_carga)

df_carga = pd.concat([resultSECO, resultNE, resultN, resultS])


new_order = ['cod_areacarga', 'din_atualizacao', 'dat_referencia', 'din_referenciautc', 'val_cargaglobal', 'val_cargaglobalcons', 'val_cargaglobalsmmgd', 'val_cargasupervisionada', 'val_carganaosupervisionada', 'val_cargammgd', 'val_consistencia']
df_carga = df_carga[new_order]
df_carga = df_carga.drop_duplicates().reset_index(drop=True)
df_carga

Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 318)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 316)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 315)
Erro ao fazer a requisição: Expecting value: line 9 column 33 (char 313)


,cod_areacarga,din_atualizacao,dat_referencia,din_referenciautc,val_cargaglobal,val_cargaglobalcons,val_cargaglobalsmmgd,val_cargasupervisionada,val_carganaosupervisionada,val_cargammgd,val_consistencia
0,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T03:00:00.000Z,35361.2270,35361.2270,35315.8360,35315.8360,0.00000,45.38990,0.0
1,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T04:00:00.000Z,34080.9905,34080.9905,34035.5995,34035.5995,0.00000,45.39010,0.0
2,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T05:00:00.000Z,32914.8085,32914.8085,32869.4195,32869.4195,0.00000,45.38975,0.0
3,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T06:00:00.000Z,32271.3705,32271.3705,32225.9795,32225.9795,0.00000,45.39100,0.0
4,SECO,2023-04-28T08:46:08.399Z,2019-04-01,2019-04-01T07:00:00.000Z,32112.6400,32112.6400,32067.2525,32067.2525,0.00000,45.38625,0.0
...,...,...,...,...,...,...,...,...,...,...,...
190175,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-01T23:00:00.000Z,13160.7880,13160.7880,13117.2080,12286.2405,830.96780,43.58000,0.0
190176,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T00:00:00.000Z,12677.8850,12677.8850,12634.3050,11803.3875,830.91725,43.58000,0.0
190177,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T01:00:00.000Z,11944.9425,11944.9425,11901.1175,11071.8195,829.29760,43.82530,0.0
190178,S,2024-09-30T03:19:28.098Z,2024-09-01,2024-09-02T02:00:00.000Z,10973.6000,10973.6000,10930.0200,10103.2540,826.76605,43.58000,0.0


Modelo final - Não são necessárias alterações

In [ ]:
import pandas as pd

failure = False; i = 2000
df_carga = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/carga_energia_di/CARGA_ENERGIA_{i}.csv'
    try:
        dados_carga = pd.read_csv(url, delimiter=';')
    except Exception as e:
        failure = True
    if i == 2000:
        df_carga = dados_carga
    elif failure == False: 
        df_carga = pd.concat([df_carga, dados_carga])
    i = i + 1
df_carga.drop(columns= 'nom_subsistema', inplace=True)
df_carga = df_carga.reset_index(drop=True)
df_carga.replace({'SE': 'SE/CO'}, inplace=True)
df_carga

,id_subsistema,din_instante,val_cargaenergiamwmed
0,N,2000-01-01,2243.512500
1,NE,2000-01-01,4646.370833
2,S,2000-01-01,4800.650000
3,SE/CO,2000-01-01,19045.995833
4,N,2000-01-02,2259.808333
...,...,...,...
36344,SE/CO,2024-11-15,NaN
36345,N,2024-11-16,NaN
36346,NE,2024-11-16,NaN
36347,S,2024-11-16,NaN


In [64]:
df_carga.to_csv('Carga_Consumo_atualizado.csv')

# ENA

ENA + MLT - Requisição de ENA está sendo feita por API, para mudar os valores da MLT, seguir os comentários em verde

In [3]:
df_mlt = pd.read_csv('Mlt_atualizado.csv')


In [4]:
df_mlt

,Unnamed: 0,Ano,(jan)MWmed,(fev)MWmed,(mar)MWmed,(abr)MWmed,(mai)MWmed,(jun)MWmed,(jul)MWmed,(ago)MWmed,...,(jul)% med.,(ago)% med.,(set)% med.,(out)% med.,(nov)% med.,(dez)% med.,(min)% med.,(jnd)% med.,(max)% med.,Subsistema
0,0,1931,69359.690000,103272.430000,107218.270000,80246.410000,55019.580000,40765.81000,31993.570000,25434.170000,...,125.101976,123.944948,132.210640,117.091602,101.535843,99.071047,60.844401,128.873962,256.490831,SE/CO
1,1,1932,69378.190000,76951.880000,68566.180000,51823.270000,40237.380000,35224.44000,27384.850000,21334.670000,...,107.080856,103.967401,91.195856,107.101383,100.543753,121.880783,42.818393,104.518598,184.086646,SE/CO
2,2,1933,79952.560000,68553.650000,58429.420000,53341.080000,38425.430000,28005.79000,21773.810000,17187.930000,...,85.140441,83.759646,82.398450,88.931164,72.262547,92.487523,38.687824,93.700592,191.264964,SE/CO
3,3,1934,57587.480000,52748.750000,52134.030000,43281.560000,30991.250000,22912.10000,18645.760000,13764.920000,...,72.909070,67.078748,70.746751,64.036409,58.436174,96.887221,32.928863,76.953285,137.762534,SE/CO
4,4,1935,72107.460000,91347.300000,80328.830000,71662.200000,44950.720000,33590.26000,25174.410000,23909.130000,...,98.437544,116.513174,107.884666,161.025812,97.634865,82.118237,50.654144,114.122940,218.523810,SE/CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,367,2019,5179.260000,3654.660000,6240.260000,6429.000000,4044.060000,2370.23000,1872.030000,1629.040000,...,49.780445,49.679218,44.599310,32.112688,24.577186,47.956864,13.207630,42.733419,82.908776,NE
368,368,2020,6104.200000,11763.360000,13678.350000,11753.790000,5592.160000,3418.92000,2444.370000,2198.020000,...,64.999934,67.030838,59.506779,57.144129,89.278287,45.942497,22.505561,74.960439,176.396836,NE
369,369,2021,6103.860000,6022.270000,9855.850000,4107.000000,2683.490000,1596.91000,1613.650000,1359.640000,...,42.909684,41.463594,43.150129,41.132936,78.305507,100.397218,16.319449,53.654954,127.321779,NE
370,370,2022,17924.000000,20879.000000,17281.000000,7449.000000,3678.000000,2857.00000,2445.000000,2056.000000,...,65.016687,62.699794,76.175769,59.951042,80.882769,97.001603,24.657269,99.432929,269.256858,NE


In [8]:
import pandas as pd


df_mlt = pd.read_csv('Mlt_atualizado.csv')
try:
    df_mlt = df_mlt.drop(columns='Unnamed: 0')
except: pass
def calcular_estatisticas_e_atualizar_porcentagens(df, novo_ano, novos_valores):
    if novo_ano in df['Ano'].values:
        print(f"O ano {novo_ano} já existe no DataFrame. Nenhuma ação será realizada.")
        return df

    df = df[df['Ano'] != 'media'].copy()

    colunas_para_remover = [col for col in df.columns if
                            'Subsistema' in col or
                            '% med.' in col or
                            '(min)' in col or
                            '(max)' in col or
                            '(jnd)' in col]
    df = df.drop(columns=colunas_para_remover, errors='ignore')

    nova_linha = {'Ano': novo_ano}
    meses = ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed',
             '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed']
    for mes, valor in zip(meses, novos_valores):
        nova_linha[mes] = valor

    df = pd.concat([df, pd.DataFrame([nova_linha])], ignore_index=True)

    df['(min)MWmed'] = df[meses].min(axis=1)
    df['(jnd)MWmed'] = df[meses].mean(axis=1)
    df['(max)MWmed'] = df[meses].max(axis=1)

    media_row = {'Ano': 'media'}
    for mes in meses:
        media_row[mes] = df[mes].mean()
    means = [df[mes].mean() for mes in meses]

    for idx in range(len(df)):
        for i, mes in enumerate(meses):
            valor = df.loc[idx, mes]
            media = means[i]
            df.loc[idx, f'{mes[0:5]}% med.'] = (valor / media) * 100 if pd.notna(valor) and media != 0 else 100

    mjdm = ['(min)MWmed', '(jnd)MWmed', '(max)MWmed']
    media_jnd = df['(jnd)MWmed'].mean()

    for idx in range(len(df)):
        for mj in mjdm:
            valor = df.loc[idx, mj]
            df.loc[idx, f'{mj[0:5]}% med.'] = (valor / media_jnd) * 100 if pd.notna(valor) and media_jnd != 0 else 100

    df = pd.concat([df, pd.DataFrame([media_row])], ignore_index=True)

    df = df.fillna(100)
    return df





# Abaixo estão os valores a serem alterados
# Altere para o ano a ser adicionado
ano = '2022'

historicoSE = df_mlt[df_mlt['Subsistema'] == 'SE/CO'].copy()
# Altere para os valores para a região SE/CO na ordem dos meses (jan, fev, mar, ..., dez)
values_inputSE = [71658, 78130, 52904,  41024, 26938, 24249, 16822, 16036, 16085, 24508, 24449, 43079]


historicoSE = calcular_estatisticas_e_atualizar_porcentagens(historicoSE, ano, values_inputSE)
historicoSE['Subsistema'] = "SE/CO"

historicoN = df_mlt[df_mlt['Subsistema'] == 'N'].copy()

 # Altere para os valores para a região N na ordem dos meses (jan, fev, mar, ..., dez)
values_inputN = [35162, 30798, 32737,  29655, 17111, 8829, 4573, 2739, 1763, 1718, 3841, 10253]


historicoN = calcular_estatisticas_e_atualizar_porcentagens(historicoN, ano, values_inputN)
historicoN['Subsistema'] = "N"


historicoS = df_mlt[df_mlt['Subsistema'] == 'S'].copy()

# Altere os valores para a região S na ordem dos meses (jan, fev, mar, ..., dez)
values_inputS = [2802, 2751, 6990,  10176, 18906, 24482, 7823, 12663, 9727, 20738, 7820, 8163]


historicoS = calcular_estatisticas_e_atualizar_porcentagens(historicoS, ano, values_inputS)
historicoS['Subsistema'] = "S"

historicoNE = df_mlt[df_mlt['Subsistema'] == 'NE'].copy()

 # Altere os valores para a região NE na ordem dos meses (jan, fev, mar, ..., dez)
values_inputNE = [17924, 20879, 17281,  7449, 3678, 2857, 2445, 2056, 2234, 1912, 4270, 9539]


historicoNE = calcular_estatisticas_e_atualizar_porcentagens(historicoNE, ano, values_inputNE)
historicoNE['Subsistema'] = "NE"

df_mlt = pd.concat([historicoSE, historicoS, historicoN, historicoNE]).reset_index(drop=True)


failure = False; i = 2000
df_earm = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ena_subsistema_di/ENA_DIARIO_SUBSISTEMA_{i}.csv'
    try:
        dados_ena = pd.read_csv(url, delimiter=';')
    except Exception as e:
        failure = True
    if i == 2000:
        df_ena = dados_ena
    elif failure == False: 
        df_ena = pd.concat([df_ena, dados_ena])
    i = i + 1
df_ena.replace({'NORDESTE': 'NE', 'NORTE': 'N','SUL': 'S','SUDESTE': 'SE/CO'}, inplace=True)
df_ena = df_ena.drop(columns=["ena_bruta_regiao_percentualmlt", "ena_armazenavel_regiao_percentualmlt"])
df_ena['ena_data'] = pd.to_datetime(df_ena['ena_data'])
df_ena['month'] = df_ena['ena_data'].dt.month
def month_value_condition(row, df, ena):
    month = row['month']
    sub = row['nom_subsistema']
    ena_value = row[ena]
    
    subsistema_columns = {
        'SE/CO': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed', 
               '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'],
        'N': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed',
              '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'],
        'NE': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed',
               '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed'],
        'S': ['(jan)MWmed', '(fev)MWmed', '(mar)MWmed', '(abr)MWmed', '(mai)MWmed', '(jun)MWmed',
              '(jul)MWmed', '(ago)MWmed', '(set)MWmed', '(out)MWmed', '(nov)MWmed', '(dez)MWmed']
    }
    
    if sub in subsistema_columns:
        month_column = subsistema_columns[sub][month - 1]  
        value = ena_value * 100 / df.loc[(df['Subsistema'] == sub) & (df['Ano'] == 'media'), month_column].values[0]
        return value
    else:
        return "Invalid subsistema"

df_ena['ena_bruta_regiao_percentualmlt'] = df_ena.apply(lambda row: month_value_condition(row, df_mlt, 'ena_bruta_regiao_mwmed'), axis=1)
df_ena['ena_armazenavel_regiao_percentualmlt'] = df_ena.apply(lambda row: month_value_condition(row, df_mlt, 'ena_armazenavel_regiao_mwmed'), axis=1)
df_ena = df_ena.drop(columns=['nom_subsistema']).reset_index(drop=True)
df_ena.replace({'SE': 'SE/CO'}, inplace=True)
df_mlt.replace({'SE': 'SE/CO'}, inplace=True)
aux = df_ena.groupby(['id_subsistema', 'month'])['ena_bruta_regiao_mwmed'].agg(['max', 'min']).reset_index()
df_ena = pd.merge(df_ena, aux, on=['id_subsistema', 'month'], how='left')
df_ena


O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.
O ano 2022 já existe no DataFrame. Nenhuma ação será realizada.


,id_subsistema,ena_data,ena_bruta_regiao_mwmed,ena_armazenavel_regiao_mwmed,month,ena_bruta_regiao_percentualmlt,ena_armazenavel_regiao_percentualmlt,max,min
0,N,2000-01-01,10392.561523,10381.552734,1,65.642536,65.573001,39296.804688,1878.000000
1,NE,2000-01-01,10174.402344,9990.510742,1,75.481418,74.117171,33141.000000,1859.000000
2,S,2000-01-01,1772.415039,1772.415039,1,23.482726,23.482726,30770.000000,1490.378052
3,SE/CO,2000-01-01,27179.791016,27097.957031,1,41.306259,41.181892,110765.000000,16210.000000
4,N,2000-01-02,10888.611328,10867.503906,1,68.775736,68.642415,39296.804688,1878.000000
...,...,...,...,...,...,...,...,...,...
36963,SE/CO,2025-04-19,46161.093750,38585.347656,4,84.431744,70.575195,69691.000000,18547.000000
36964,N,2025-04-20,20001.433594,13944.609375,4,73.641919,51.341710,34472.722656,5722.000000
36965,NE,2025-04-20,3235.050049,3213.600098,4,28.258934,28.071563,25125.000000,1828.000000
36966,S,2025-04-20,4593.528809,4157.463867,4,70.184455,63.521826,61485.000000,751.440002


Exportar Ena e MLT

In [2]:
df_ena.to_csv('Ena_atualizado.csv')
df_mlt.to_csv("Mlt_atualizado.csv")

# EARM

Atualizar a EARM - Não são necessárias alterações

In [ ]:
import pandas as pd

failure = False; i = 2000
df_earm = pd.DataFrame
while failure == False:
    url = f'https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ear_subsistema_di/EAR_DIARIO_SUBSISTEMA_{i}.csv'
    try:
        dados_earm = pd.read_csv(url, delimiter=';')
    except Exception as e:
        failure = True
    if i == 2000:
        df_earm = dados_earm
    elif failure == False: 
        df_earm = pd.concat([df_earm, dados_earm])
    i = i + 1
df_earm.drop(columns= 'nom_subsistema', inplace=True)
sum_seco = df_earm[df_earm['id_subsistema'] == 'SE']
sum_s = df_earm[df_earm['id_subsistema'] == 'S']
sum_ne = df_earm[df_earm['id_subsistema'] == 'NE']
sum_n = df_earm[df_earm['id_subsistema'] == 'N']
sum_ear_verif_subsistema_mwmes = (sum_seco['ear_verif_subsistema_mwmes'].iloc[-1] + sum_s['ear_verif_subsistema_mwmes'].iloc[-1] +
sum_ne['ear_verif_subsistema_mwmes'].iloc[-1] +sum_n['ear_verif_subsistema_mwmes'].iloc[-1])
sum_ear_max_subsistema = (sum_seco['ear_max_subsistema'].iloc[-1] + sum_s['ear_max_subsistema'].iloc[-1] +
sum_ne['ear_max_subsistema'].iloc[-1] +sum_n['ear_max_subsistema'].iloc[-1])


df_earm['verif_max_ratio'] = (sum_ear_verif_subsistema_mwmes / sum_ear_max_subsistema)*100

df_earm.reset_index(drop=True, inplace=True)
df_earm.replace({'SE': 'SE/CO'}, inplace=True)
df_earm

,id_subsistema,ear_data,ear_max_subsistema,ear_verif_subsistema_mwmes,ear_verif_subsistema_percentual,verif_max_ratio
0,NE,2000-01-01,49967.000000,11171.000000,22.360001,69.681548
1,N,2000-01-01,12311.000000,7656.000000,62.189999,69.681548
2,SE/CO,2000-01-01,155759.000000,27853.000000,17.879999,69.681548
3,S,2000-01-01,13976.000000,7094.000000,50.759998,69.681548
4,NE,2000-01-02,49967.000000,11395.000000,22.809999,69.681548
...,...,...,...,...,...,...
36687,S,2025-02-09,20459.242188,12309.364258,60.165298,69.681548
36688,NE,2025-02-10,51691.226562,39521.175781,76.456299,69.681548
36689,N,2025-02-10,15302.396484,13833.227539,90.399101,69.681548
36690,SE/CO,2025-02-10,204615.328125,137957.640625,67.422897,69.681548


Exportar a EARM

In [9]:
df_earm.to_csv('EARM_atualizado.csv')

# PLD

Atualizar PLD

In [2]:
pldH = pd.read_csv('PLD Horário Comercial Historico.csv')
pldH

,Unnamed: 0,Hora,Submercado,Data,Valor
0,0,0,N,2001-06-30,684
1,1,1,N,2001-06-30,684
2,2,2,N,2001-06-30,684
3,3,3,N,2001-06-30,684
4,4,4,N,2001-06-30,684
...,...,...,...,...,...
815731,815731,19,SECO,2024-10-04,643
815732,815732,20,SECO,2024-10-04,"639,75"
815733,815733,21,SECO,2024-10-04,"634,11"
815734,815734,22,SECO,2024-10-04,"624,42"


In [ ]:
import pandas as pd

pld = pd.read_csv('modelo_pld.csv', delimiter = ";") # ***Arquivo a ser adicionado, é necessário apenas mudar o nome do arquivo modelo_pld***

pld = pld.melt(id_vars=['Hora', 'Submercado'], var_name='Data', value_name='Valor')

pld['Data'] = pd.to_datetime(pld['Data'], format='%d/%m/%Y')

submercados = {
    'NORTE': 'N',
    'SUDESTE': 'SE/CO',
    'SUL': 'S',
    'NORDESTE': 'NE'
}
pld['Submercado'] = pld['Submercado'].replace(submercados)

pld['Data'] = pd.to_datetime(pld['Data'], format='%d/%m/%Y').dt.strftime('%Y-%m-%d')
pld['Valor'] = pld['Valor'].str.replace('.', '', regex=False)
pld['Valor'] = pld['Valor'].str.replace(',', '.', regex=False)
pldH = pd.read_csv('PLD Horário Comercial Historico.csv')
pldH = pldH[['Hora', 'Submercado', 'Data', 'Valor']]
pld = pd.concat([pldH, pld]).drop_duplicates(subset=['Hora', 'Submercado', 'Data'], keep='first')
pld['Submercado'] = pld['Submercado'].astype(str)
pldn = pld[pld['Submercado'] == 'N'].reset_index(drop=True)
pldne = pld[pld['Submercado'] == 'NE'].reset_index(drop=True)
plds = pld[pld['Submercado'] == 'S'].reset_index(drop=True)
pldseco = pld[pld['Submercado'] == 'SE/CO'].reset_index(drop=True)
pld = pd.concat([pldseco, plds, pldne, pldn]).reset_index(drop=True)
pld['Valor'] = pd.to_numeric(pld['Valor'], errors='coerce')

pld



C:\Users\Fernando\AppData\Local\Temp\ipykernel_35320\2405683548.py:25: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  pldH = pd.read_csv('PLD Horário Comercial Historico.csv')


,Hora,Submercado,Data,Valor
0,0,SE/CO,2001-06-30,684.00
1,1,SE/CO,2001-06-30,684.00
2,2,SE/CO,2001-06-30,684.00
3,3,SE/CO,2001-06-30,684.00
4,4,SE/CO,2001-06-30,684.00
...,...,...,...,...
815731,19,N,2024-10-04,644.90
815732,20,N,2024-10-04,642.68
815733,21,N,2024-10-04,634.13
815734,22,N,2024-10-04,624.43


Substituição de Célula

In [ ]:
import pandas as pd
pld = pd.read_csv("PLD Horário Comercial Historico.csv")
# AVISO!!! Rodar essa célula vai descartar as mudanças feitas nas células anteriores do PLD

In [ ]:
def update_value(df, hora, submercado, data, new_value):
    condition = (df['Hora'] == hora) & (df['Submercado'] == submercado) & (df['Data'] == data)
    if df[condition].empty:
        print("No matching row found.")
    else:
        df.loc[condition, 'Valor'] = new_value
        print(f"Updated 'Valor' for Hora={hora}, Submercado={submercado}, Data={data} to {new_value}.")

update_value(pld, 1, 'N', '2001-06-30', 1,700.32) 
# ***Tire a # da linha acima para rodar, linhas em verde são apenas comentários e não são processadas pelo código***
#formato: (pld, hora, 'submercado', 'YYYY-MM-DD', valor correto (com ',' como separador de milhar e '.' como separador decimal))



Exportar a PLD

In [ ]:
pld.to_csv('PLD Horário Comercial Historico.csv')

# Tarifas

Atualizar Tarifas

In [ ]:
import pandas as pd
tarifa = pd.read_excel("Histórico Tarifas Distribuidoras (desde 2010).xlsx") # ***Mudar pelo nome do arquivo atualizado***
tarifa = tarifa[tarifa['Base Tarifária'] == 'Tarifa de Aplicação']
tarifa = tarifa[(tarifa['Modalidade'] == 'Azul') | (tarifa['Modalidade'] == 'Verde')]
tarifa = tarifa[(tarifa['Detalhe'] == 'Não se aplica') | (tarifa['Detalhe'] == 'APE')]
tarifa = tarifa[tarifa['Classe'] == 'Não se aplica']
tarifa = tarifa[tarifa['Subclasse'] == 'Não se aplica']
tarifa = tarifa[tarifa['Acessante'] == 'Não se aplica']
tarifa = tarifa[tarifa['Subgrupo'].str.contains("A", na=False)]
tarifa['Modalidade'] = tarifa.apply(
    lambda row: f"{row['Modalidade']} {row['Detalhe']}" if row['Detalhe'] == 'APE' else row['Modalidade'] 
    , axis=1)
tarifa = tarifa.drop(columns=['Resolução ANEEL', 'Classe', 'Subclasse', 'Acessante', 'Base Tarifária', 'Detalhe'])
tarifa

,Sigla,Início Vigência,Fim Vigência,Subgrupo,Modalidade,Posto,Unidade,TUSD,TE
1,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul APE,Fora ponta,R$/kW,5.01,0.00
3,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta,R$/kW,5.01,0.00
5,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta,R$/MWh,16.67,0.00
7,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta seca,R$/MWh,16.67,144.52
9,Amazonas Energia,2010-11-01,2011-10-31,A3,Azul,Fora ponta úmida,R$/MWh,16.67,129.11
...,...,...,...,...,...,...,...,...,...
252779,Sulgipe,2024-05-22,2025-05-21,A4,Verde,Fora ponta,R$/MWh,63.81,214.42
252782,Sulgipe,2024-05-22,2025-05-21,A4,Verde APE,Não se aplica,R$/kW,22.80,0.00
252784,Sulgipe,2024-05-22,2025-05-21,A4,Verde,Não se aplica,R$/kW,22.80,0.00
252786,Sulgipe,2024-05-22,2025-05-21,A4,Verde APE,Ponta,R$/MWh,1539.98,0.00


Atualizar Bandeiras

In [ ]:
import pandas as pd

bandeiras = pd.read_excel("2024 09 - HISTÓRICO BANDEIRAS TARIFÁRIAS TODOS SUBMERCADOS (jan2014~).xlsx") # ***Mudar pelo nome do arquivo atualizado***

mid_row = bandeiras.shape[0] // 2
mid_col = bandeiras.shape[1] // 2

df_top_left = bandeiras.iloc[:mid_row, :mid_col]
df_top_right = bandeiras.iloc[:mid_row, mid_col:]
df_bottom_left = bandeiras.iloc[mid_row:, :mid_col]
df_bottom_right = bandeiras.iloc[mid_row:, mid_col:]


def bottom(df):
    df_processed = df.iloc[1:, 1:]
    
    df_transposed = df_processed.transpose()
    
    df_transposed.columns = df_transposed.iloc[0]
    df_final = df_transposed[1:]  
    
    return df_final.reset_index(drop=True)

def top(df):
    df_processed = df.iloc[:13, 1:]
    
    df_transposed = df_processed.transpose()
    
    df_transposed.columns = df_transposed.iloc[0]
    df_final = df_transposed[1:]  
    
    return df_final.reset_index(drop=True)

bandeiraSeco = top(df_top_left)
bandeiraNordeste = top(df_top_right)
bandeiraSul = bottom(df_bottom_left)
bandeiraNorte = bottom(df_bottom_right)
bandeiraNorte


Unnamed: 14,NORTE,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,2014,Amarela,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
1,2015,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
2,2016,Vermelha,Vermelha,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Amarela,Verde
3,2017,Verde,Verde,Amarela,Vermelha 1,Vermelha 1,Verde,Amarela,Vermelha 1,Amarela,Vermelha 2,Vermelha 2,Vermelha 1
4,2018,Verde,Verde,Verde,Verde,Amarela,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Amarela,Verde
5,2019,Verde,Verde,Verde,Verde,Amarela,Verde,Amarela,Vermelha 1,Vermelha 1,Amarela,Vermelha 1,Amarela
6,2020,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Vermelha 2
7,2021,Amarela,Amarela,Amarela,Amarela,Vermelha 1,Vermelha 2,Vermelha 2,Vermelha 2,Escassez,Escassez,Escassez,Escassez
8,2022,Escassez,Escassez,Escassez,Escassez,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde
9,2023,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde


Substituir valores

In [ ]:
def substituir_valor(df, ano, mes, novo_valor):
    # Mapeia os meses para suas respectivas colunas
    meses_map = {
        'JAN': 1,
        'FEV': 2,
        'MAR': 3,
        'ABR': 4,
        'MAI': 5,
        'JUN': 6,
        'JUL': 7,
        'AGO': 8,
        'SET': 9,
        'OUT': 10,
        'NOV': 11,
        'DEZ': 12
    }

    # Verifica se o ano já está no DataFrame
    if ano in df.iloc[:, 0].values:
        # Encontra o índice do ano
        idx = df[df.iloc[:, 0] == ano].index[0]

        # Verifica se o mês está mapeado
        if mes in meses_map:
            # Atualiza o valor na célula específica
            coluna = meses_map[mes]
            df.iat[idx, coluna] = novo_valor
        else:
            print("Mês inválido")
            return df
    else:
        # Se o ano não está presente, adiciona uma nova linha
        nova_linha = [ano] + [0] * (df.shape[1] - 1)  # Cria nova linha com zeros
        df.loc[len(df)] = nova_linha  # Adiciona a nova linha
        idx = len(df) - 1  # Atualiza o índice da nova linha

        # Verifica se o mês está mapeado
        if mes in meses_map:
            coluna = meses_map[mes]
            df.iat[idx, coluna] = novo_valor
        else:
            print("Mês inválido")
            return df
    
    return df

bandeiraSeco = substituir_valor(bandeiraSeco, 2024, 'DEZ', 'oi') # dataframe a ser alterada, yyyy, 'MMM', 'valor adicionado'
# ***Tire apenas a primeira # da linha acima para rodar, linhas em verde são apenas comentários e não são processadas pelo código***

Unnamed: 1,SE/CO,JAN,FEV,MAR,ABR,MAI,JUN,JUL,AGO,SET,OUT,NOV,DEZ
0,2014,Amarela,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
1,2015,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha,Vermelha
2,2016,Vermelha,Vermelha,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Amarela,Verde
3,2017,Verde,Verde,Amarela,Vermelha 1,Vermelha 1,Verde,Amarela,Vermelha 1,Amarela,Vermelha 2,Vermelha 2,Vermelha 1
4,2018,Verde,Verde,Verde,Verde,Amarela,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Vermelha 2,Amarela,Verde
5,2019,Verde,Verde,Verde,Verde,Amarela,Verde,Amarela,Vermelha 1,Vermelha 1,Amarela,Vermelha 1,Amarela
6,2020,Amarela,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Vermelha 2
7,2021,Amarela,Amarela,Amarela,Amarela,Vermelha 1,Vermelha 2,Vermelha 2,Vermelha 2,Escassez,Escassez,Escassez,Escassez
8,2022,Escassez,Escassez,Escassez,Escassez,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde
9,2023,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde,Verde


Exporta as Bandeiras

In [44]:
bandeiraNorte.to_csv('bandeira_N.csv')
bandeiraSul.to_csv('bandeira_S.csv')
bandeiraNordeste.to_csv('bandeira_NE.csv')
bandeiraSeco.to_csv('bandeira_SECO.csv')

Exporta as Tarifas

In [39]:
tarifa.to_csv("tarifa_atualizado.csv")

# Curva Forward

In [ ]:
import pandas as pd

df = pd.read_excel("DADOS CURVA FORWARD CONSOLIDADOS.xlsx") # Mudar pelo nome do arquivo atualizado

def soma_base(df, dia, subregiao, a_plus):
    # Converter o dia para o formato datetime
    dia = pd.to_datetime(dia)

    # Filtrar o DataFrame pelo dia
    df_filtrado = df[(df['DIA'] == dia)]

    if df_filtrado.empty:
        return f"Não há dados para a data {dia.date()}."

    # Definir a coluna com base na subregião e A+
    if subregiao.lower() == 'norte':
        coluna_base = f'Norte Convencional {a_plus}'
    elif subregiao.lower() == 'sul':
        coluna_base = f'Sul Convencional {a_plus}'
    elif subregiao.lower() == 'nordeste':
        coluna_base = f'Nordeste Convencional {a_plus}'
    elif subregiao.lower() == 'se/co':
        coluna_base = f'SE/CO Convencional {a_plus}'
    else:
        return "Subregião inválida."

    # Verificar se a coluna existe
    if coluna_base not in df.columns:
        return f"A coluna {coluna_base} não existe no DataFrame."
    
    # Pegar o valor da coluna SE/CO Convencional {a_plus}
    valor_base = df_filtrado[f'SE/CO Convencional {a_plus}'].iloc[0]

    # Calcular a soma
    soma = df_filtrado[coluna_base].sum()

    # Arredondar o resultado para 2 casas decimais
    resultado = round(soma + valor_base, 2)
    
    return resultado


df_organizado = df[['DIA', 'SEMANA', 'MÊS', 
                    'SE/CO CONVENCIONAL', 
                    'Unnamed: 4',  
                    'Unnamed: 5',  
                    'Unnamed: 6', 
                    'SUL CONVENCIONAL', 
                    'Unnamed: 8',  
                    'Unnamed: 9',  
                    'Unnamed: 10',
                    'NORDESTE CONVENCIONAL', 
                    'Unnamed: 12',  
                    'Unnamed: 13',  
                    'Unnamed: 14', 
                    'NORTE CONVENCIONAL', 
                    'Unnamed: 16',  
                    'Unnamed: 17',  
                    'Unnamed: 18',
                    'SE/CO INCENTIVADA',
                    'Unnamed: 20',  
                    'Unnamed: 21',  
                    'Unnamed: 22']]

df_organizado.columns = ['DIA', 'SEMANA', 'MÊS', 
                         'SE/CO Convencional A+1', 
                         'SE/CO Convencional A+2', 
                         'SE/CO Convencional A+3', 
                         'SE/CO Convencional A+4', 
                         'Sul Convencional A+1', 
                         'Sul Convencional A+2', 
                         'Sul Convencional A+3', 
                         'Sul Convencional A+4',
                         'Nordeste Convencional A+1', 
                         'Nordeste Convencional A+2', 
                         'Nordeste Convencional A+3', 
                         'Nordeste Convencional A+4',
                         'Norte Convencional A+1', 
                         'Norte Convencional A+2', 
                         'Norte Convencional A+3', 
                         'Norte Convencional A+4',
                         'SE/CO Incentivada A+1',
                         'SE/CO Incentivada A+2',
                         'SE/CO Incentivada A+3',
                         'SE/CO Incentivada A+4']

def aplicar_soma_base(row):
    for col in row.index[3:]: 
        if col not in ['SE/CO Convencional A+1', 'SE/CO Convencional A+2', 'SE/CO Convencional A+3', 'SE/CO Convencional A+4', 
                       'SE/CO Incentivada A+1', 'SE/CO Incentivada A+2', 'SE/CO Incentivada A+3', 'SE/CO Incentivada A+4']:
            subregiao, a_plus = col.split(" ")[0].lower(), col.split(" ")[-1]
            valor = soma_base(df_organizado, row['DIA'], subregiao, a_plus)
            row[col] = valor  
    return row

def aplicar_soma_incentivada(row):
    """
    Soma os valores da coluna Incentivada nas colunas de Convencional.
    """
    for subregiao in ['sul', 'norte', 'nordeste']:
        for a_plus in ['A+1', 'A+2', 'A+3', 'A+4']:
            col_conv = f"{subregiao.capitalize()} Convencional {a_plus}"
            col_incent = f"SE/CO Incentivada {a_plus}"
            
            if col_conv in row.index and col_incent in row.index:
                row[col_conv] = round(row[col_conv] + row[col_incent], 2)

    return row


df_organizado = df_organizado.drop(index=0).reset_index(drop=True)

convencional = df_organizado.apply(aplicar_soma_base, axis=1)

incentivada = convencional.apply(aplicar_soma_incentivada, axis=1)
convencional = convencional.iloc[:, :-4]
incentivada = incentivada.drop(columns=['SE/CO Convencional A+1', 'SE/CO Convencional A+2', 'SE/CO Convencional A+3', 'SE/CO Convencional A+4'])
incentivada.columns = incentivada.columns.str.replace('Convencional', 'Incentivada')
colunas_seco_incentivada = ['SE/CO Incentivada A+1', 'SE/CO Incentivada A+2', 
                                'SE/CO Incentivada A+3', 'SE/CO Incentivada A+4']
    
colunas_outros = [col for col in incentivada.columns if col not in colunas_seco_incentivada and col not in ['DIA', 'SEMANA', 'MÊS']]

colunas_final = ['DIA', 'SEMANA', 'MÊS'] + colunas_seco_incentivada + colunas_outros
incentivada = incentivada[colunas_final]

Exportar Curvas

In [61]:
convencional.to_csv("Curva_convencional.csv")
incentivada.to_csv("Curva_incentivada.csv")